In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate as cv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import cross_validate
import random
from datetime import datetime
from sklearn.model_selection import KFold
from collections import defaultdict
#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

In [22]:
def get_top5_recommendations(predictions, topN = 5):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [33]:
ratings=pd.read_csv('ml-25m/ratings.csv',sep=',')
movies= pd.read_csv('ml-25m/movies.csv')
ratings['timestamp'] = ratings['timestamp'].apply(datetime.fromtimestamp)
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,2006-05-17 10:34:04
1,1,306,3.5,2006-05-17 07:26:57
2,1,307,5.0,2006-05-17 07:27:08
3,1,665,5.0,2006-05-17 10:13:40
4,1,899,3.5,2006-05-17 07:21:50
...,...,...,...,...
25000090,162541,50872,4.5,2009-04-28 16:16:12
25000091,162541,55768,2.5,2009-04-28 15:53:18
25000092,162541,56176,2.0,2009-04-28 15:31:37
25000093,162541,58559,4.0,2009-04-28 16:17:14


In [34]:
movie_data_ratings_data=movies.merge(ratings,on = 'movieId',how = 'inner')
movie_data_ratings_data.head(3)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5,2006-03-03 14:57:00
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,2015-08-13 08:23:35
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0,2019-11-16 17:44:12


In [35]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')

In [25]:
# Calculate count rating of all movies
movie_data_ratings_data.groupby('title')['rating'].count().sort_values(ascending=False).head()

title
Forrest Gump (1994)                 81491
Shawshank Redemption, The (1994)    81482
Pulp Fiction (1994)                 79672
Silence of the Lambs, The (1991)    74127
Matrix, The (1999)                  72674
Name: rating, dtype: int64

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

(62423, 191)

#Cosine similarity

In [7]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.32107747, 0.06609706, 0.05824133],
       [0.32107747, 1.        , 0.        , 0.        ],
       [0.06609706, 0.        , 1.        , 0.36875378],
       [0.05824133, 0.        , 0.36875378, 1.        ]])

In [76]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Ver las recomendaciones top

In [9]:
genre_recommendations('Good Will Hunting (1997)').head(20)

24                            Leaving Las Vegas (1995)
27                                   Persuasion (1995)
34                                   Carrington (1995)
45                How to Make an American Quilt (1995)
48                        When Night Is Falling (1995)
73                                 Bed of Roses (1996)
82     Once Upon a Time... When We Were Colored (1995)
84                           Angels and Insects (1995)
103              Bridges of Madison County, The (1995)
129                           Frankie Starlight (1995)
138                       Up Close and Personal (1996)
177                                    Mad Love (1995)
180                     Moonlight and Valentino (1995)
189                         Scarlet Letter, The (1995)
200                               Total Eclipse (1995)
205                       Walk in the Clouds, A (1995)
213                              Before Sunrise (1995)
219                           Circle of Friends (1995)
246       

In [10]:
genre_recommendations('Toy Story (1995)').head(20)

2203                                           Antz (1998)
3021                                    Toy Story 2 (1999)
3653        Adventures of Rocky and Bullwinkle, The (2000)
3912                      Emperor's New Groove, The (2000)
4780                                 Monsters, Inc. (2001)
9949     DuckTales: The Movie - Treasure of the Lost La...
10773                                     Wild, The (2006)
11604                               Shrek the Third (2007)
12969                       Tale of Despereaux, The (2008)
17431    Asterix and the Vikings (Astérix et les Viking...
20015                                         Turbo (2013)
22286                                       Aladdin (1992)
22353                                Boxtrolls, The (2014)
22633            Toy Story Toons: Hawaiian Vacation (2011)
22634                    Toy Story Toons: Small Fry (2011)
23212                             The Magic Crystal (2011)
28245                                Brother Bear 2 (200

In [11]:
genre_recommendations('Saving Private Ryan (1998)').head(20)

460                         Heaven & Earth (1993)
1177                        Apocalypse Now (1979)
1201                 Boot, Das (Boat, The) (1981)
1838        All Quiet on the Western Front (1930)
1939                   Saving Private Ryan (1998)
2336                    Thin Red Line, The (1998)
2852                      Dirty Dozen, The (1967)
2969                      Longest Day, The (1962)
2973                     Tora! Tora! Tora! (1970)
3345                              Red Dawn (1984)
3421                Force 10 from Navarone (1978)
3544                 Fighting Seabees, The (1944)
3652                          Patriot, The (2000)
4852                    Behind Enemy Lines (2001)
4895                                Sahara (1943)
4904                       Black Hawk Down (2001)
5046                      We Were Soldiers (2002)
5312                           Windtalkers (2002)
5787                                  Zulu (1964)
5803    Victory (a.k.a. Escape to Victory) (1981)


In [12]:
## Filtrado colaborativo

In [38]:
# Fill NaN values in user_id and movie_id column with 0
ratings['userId'] = ratings['userId'].fillna(0)
ratings['movieId'] = ratings['movieId'].fillna(0)

# Replace NaN values in rating column with average of all values
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,2006-05-17 10:34:04
1,1,306,3.5,2006-05-17 07:26:57
2,1,307,5.0,2006-05-17 07:27:08
3,1,665,5.0,2006-05-17 10:13:40
4,1,899,3.5,2006-05-17 07:21:50
...,...,...,...,...
25000090,162541,50872,4.5,2009-04-28 16:16:12
25000091,162541,55768,2.5,2009-04-28 15:53:18
25000092,162541,56176,2.0,2009-04-28 15:31:37
25000093,162541,58559,4.0,2009-04-28 16:17:14


In [9]:
ratings=ratings.sort_values(by='timestamp')

In [45]:
reviews=ratings[['userId','movieId','rating','timestamp']].sort_values(by='timestamp').copy()
ratings['user_id_simple'] = pd.factorize(reviews.userId)[0]
ratings['movie_id_simple'] = pd.factorize(reviews.movieId)[0]
reviews['user_id_simple'] = pd.factorize(reviews.userId)[0]
reviews['movie_id_simple'] = pd.factorize(reviews.movieId)[0]
reviews.tail(10)

,userId,movieId,rating,timestamp,user_id_simple,movie_id_simple
13207853,85523,59315,4.0,2019-11-21 04:09:53,162540,9856
13207872,85523,122912,4.5,2019-11-21 04:09:59,162540,25055
13207855,85523,60069,4.5,2019-11-21 04:10:06,162540,9920
13207885,85523,195165,3.0,2019-11-21 04:10:24,162540,51847
13207868,85523,109487,4.5,2019-11-21 04:10:45,162540,21683
13207880,85523,168252,4.0,2019-11-21 04:11:19,162540,38256
1284508,8642,202101,2.0,2019-11-21 04:11:52,103213,55366
13207893,85523,204704,4.0,2019-11-21 04:12:13,162540,56598
1284464,8642,122914,4.0,2019-11-21 04:12:29,103213,23909
13207877,85523,149406,4.5,2019-11-21 04:15:03,162540,30127


In [11]:
reviews.shape
reviews

,userId,movieId,rating,user_id_simple,movie_id_simple
326761,2262,21,3.0,0,0
326810,2262,1079,3.0,0,1
326767,2262,47,5.0,0,2
15845015,102689,1,4.0,1,3
15845023,102689,39,5.0,1,4
...,...,...,...,...,...
13207880,85523,168252,4.0,162540,38256
1284508,8642,202101,2.0,103213,55366
13207893,85523,204704,4.0,162540,56598
1284464,8642,122914,4.0,103213,23909


In [12]:
# getting the number unique users and restaurants
unique_users = reviews.userId.unique().shape[0]
unique_restaurants = reviews.movieId.unique().shape[0]

reader = Reader( rating_scale = ( 0, 5 ) )
reviews.head()

,userId,movieId,rating,user_id_simple,movie_id_simple
326761,2262,21,3.0,0,0
326810,2262,1079,3.0,0,1
326767,2262,47,5.0,0,2
15845015,102689,1,4.0,1,3
15845023,102689,39,5.0,1,4


In [22]:
# Randomly sample dataset too big
small_data = reviews.sample(frac=0.0010)
# Check the sample info
small_data= small_data[['user_id_simple', 'movie_id_simple', 'rating']]
small_data

,user_id_simple,movie_id_simple,rating
4509235,83996,4091,3.0
13020005,14975,127,3.0
22300239,68127,2220,5.0
13900712,135958,237,4.0
19848246,36169,1159,3.0
...,...,...,...
8319143,146080,34609,3.0
21612408,29212,1166,4.0
1554911,9859,291,3.0
7175707,158194,81,5.0


In [66]:
surprise_dataset = Dataset.load_from_df( small_data[ ["user_id_simple","movie_id_simple","rating"] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.3)

In [71]:

sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud user_user
               }
for i in (3,50,100):
    algo1 = KNNBasic(k=i, min_k=3, sim_options=sim_options)
    algo1.fit(train_set)
    predictions1 = algo1.test(test_set)
    print(accuracy.rmse(predictions1))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0610
1.0610382495814323
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0610
1.0610382495814323
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0610
1.0610382495814323


In [23]:
top5_recommendations = get_top5_recommendations(predictions1)
#top5_recommendations
predictions1

[Prediction(uid=32897, iid=4437, r_ui=3.5, est=3.5323714285714285, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=161370, iid=270, r_ui=4.0, est=3.5323714285714285, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=63466, iid=3761, r_ui=5.0, est=3.5323714285714285, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=118635, iid=8603, r_ui=2.5, est=3.5323714285714285, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=152085, iid=6504, r_ui=3.5, est=3.5323714285714285, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=137098, iid=8187, r_ui=4.0, est=3.5323714285714285, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=160514, iid=54120, r_ui=2.0, est=3.5323714285714285, details={'was_impossible': True, 'reason': 'User and/or item is unknown

In [1]:

# importing the module
import imdb
   
# creating instance of IMDb
ia = imdb.IMDb()
   
# getting top 250 movies
search = ia.get_top250_movies()
  
# printing only first 10 movies title
for i in range(10):
    print(search[i]['title'])

# get a movie and print its director(s)
the_matrix = ia.get_movie('0133093')
for director in the_matrix['directors']:
    print(director['name'])

# show all information that are currently available for a movie
print(sorted(the_matrix.keys()))

# show all information sets that can be fetched for a movie
print(ia.get_movie_infoset())

# update a Movie object with more information
ia.update(the_matrix, ['technical'])
# show which keys were added by the information set
print(the_matrix.infoset2keys['technical'])
# print one of the new keys
print(the_matrix.get('tech'))    

The Shawshank Redemption
The Godfather
The Godfather: Part II
The Dark Knight
12 Angry Men
Schindler's List
The Lord of the Rings: The Return of the King
Pulp Fiction
The Good, the Bad and the Ugly
The Lord of the Rings: The Fellowship of the Ring
Lana Wachowski
Lilly Wachowski
['akas', 'animation department', 'art department', 'art directors', 'aspect ratio', 'assistant directors', 'box office', 'camera department', 'canonical title', 'cast', 'casting department', 'casting directors', 'certificates', 'cinematographers', 'color info', 'composers', 'costume departmen', 'costume designers', 'countries', 'country codes', 'cover url', 'director', 'directors', 'distributors', 'editorial department', 'editors', 'full-size cover url', 'genres', 'imdbID', 'kind', 'language codes', 'languages', 'localized title', 'location management', 'long imdb canonical title', 'long imdb title', 'make up department', 'miscellaneous', 'music department', 'original air date', 'other companies', 'plot', 'plot 

In [8]:
the_matrix = ia.get_movie('0133093')
for director in the_matrix['directors']:
    print(director['name'])

Lana Wachowski
Lilly Wachowski


In [ ]:
##Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [62]:
#predictions1
predictions_list = []
for x in predictions1:
    predictions_list.append((x.uid, x.iid, round(x.est,1)))
#test_set
#predictions_list

In [64]:
i =0
for x in predictions_list:
    for y in test_set:
    # first number is the user id
        if x[0] == y[0]:
            #print(x[0])
            i = i +1
    if i ==3:
        break
    

32897
32897
32897
32897
161370
161370
63466
63466
118635
152085
137098
160514
160514
95050
131167
101035
101035
78654
143942
93935
93935
159161
6205
61403
31973
159658
101721
75356
153167
99115
88423
49725
49725
153559
153559
146336
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
40666
44840
160541
89784
105596
41339
136047
87087
158864
69634
160225
22230
22230
135381
94816
68008
72170
135851
57262
32771
103491
42443
112402
10549
104849
156128
108284
144724
144724
53460
153959
146338
128428
4678
86151
20011
33324
21962
119594
125124
138222
100704
100704
103915
103915
86198
143284
41690
62166
116850
34388
118920
34448
50225
50225
80730
62971
100261
100261
148175
148175
148175
148175
133174
72510
102075
30129
41353
81739
87721
87721
48424
68942
76372
135765
97822
60702
9202
17046
102173
36384
43782
129267
69963
39756
45864
45864
55809
9061
105623
104899
110112
110112
153384
92709
92709
157488
74711
74711
81010
127020
50346
64937
35256
155934
155934
105

36563
79807
79807
120071
135207
112065
112065
153404
142612
78815
70034
84844
84844
122122
122122
35224
143817
97216
116
96676
155614
51491
47618
95955
51549
51549
71525
70535
54374
52339
88833
88833
104207
64232
73032
4982
133203
123085
78537
102984
68168
136623
136623
136623
136623
136623
6857
11274
44289
14485
156009
114032
145642
145642
61441
90195
152460
26041
50716
5363
152102
73464
73464
73464
114203
114203
63003
51789
51789
152149
32589
21668
42631
48146
139851
145855
4222
158352
61263
61263
161685
49379
14324
135759
84223
129983
80431
156691
156125
44506
20778
122370
121864
140773
30607
8158
141396
46713
162109
12644
86753
86753
64904
120586
145284
83802
21053
140639
114515
117362
16622
92125
113182
47479
82460
41457
11810
34167
23534
145477
144002
140849
76917
76917
145483
76773
14178
79618
145238
79169
50762
71164
52295
150156
150156
62520
62520
43349
104059
121957
121957
85768
123196
60762
70889
127639
127639
15622
67497
64321
92224
152796
6307
156899
122075
99008
101031
12

36123
66929
141431
141431
78705
107415
125261
87724
21190
60877
987
122744
122744
69383
86753
86753
38857
69178
153256
140768
18531
52836
52836
64774
76006
60414
121979
81510
81510
81510
161991
161991
161991
141673
32482
22399
117855
108747
77234
46155
103902
49695
62929
48546
137473
133527
131545
124084
12555
910
96262
96262
39254
130074
45625
85922
65466
85370
136623
136623
136623
136623
136623
60282
135631
82611
4443
77319
86592
91310
120896
120896
159423
101088
101225
57581
115225
72790
54518
29497
98769
960
95875
95875
64725
128392
63089
86706
40101
107919
77047
56433
3382
137668
38573
156100
140073
22243
22243
57011
88833
88833
688
161966
159071
36893
33888
15212
118142
1823
75857
122367
71703
75844
37767
46465
125342
151703
94731
94731
96328
22243
22243
57319
113554
34573
31251
55646
55646
120861
51337
157282
64127
143857
143857
114123
59452
34017
71942
125758
125758
131453
96666
162015
28642
17931
70118
107232
144471
88764
43598
90671
57996
44237
44237
44237
78918
18671
126515


121186
104716
73728
55820
85921
148936
148936
159378
159378
32438
32438
32438
55097
44811
105113
24549
56138
56138
56138
56138
56138
56138
55816
33280
58554
75155
67121
67121
69649
97586
45269
45269
124237
120896
120896
129166
27728
5866
120584
159241
159241
127448
91536
132142
132142
57882
47481
596
151841
59905
113788
61788
74302
53712
99914
78444
90285
160224
21810
158489
23356
68848
71875
155760
136623
136623
136623
136623
136623
116353
77688
64718
80142
149883
150208
35226
35226
146284
102887
56687
36777
38124
30548
61021
153662
84790
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
97875
30738
30738
74434
124961
150354
48526
86302
72146
3339
43257
42399
64383
112312
157628
59597
76154
94519
56533
26048
120793
147397
48020
146875
119309
119309
102249
154927
79449
79449
61276
94466
13697
140777
47393
37622
45839
45839
54417
136622
53542
81550
57619
113985
113985
117887
53543
141066
138194
147793
56902
56022
42838
146796
71177
110408
103761
123061


15514
117924
80270
64151
46238
2111
54872
47601
20299
151689
114084
33382
14984
105281
49705
115468
62102
50686
77753
141626
58339
280
79987
133292
63432
147312
147312
144470
35232
115590
100181
100181
115239
115239
58422
62876
89868
52218
159851
87147
112684
145221
128156
72321
43336
43336
125097
125097
53597
64131
158448
116636
144089
99748
62055
132862
61109
70729
154366
139061
121757
135844
31333
46943
116348
116348
109420
37570
78435
141832
54339
54339
82158
82158
36804
68014
68014
142498
81919
152087
77332
100704
100704
73933
73588
26408
80969
128779
18664
85058
129195
91723
67714
134053
143228
89842
39467
112040
114948
27948
145288
122434
111994
47223
145058
145058
82302
153034
10523
67002
67002
134844
52786
131986
103169
128534
154530
154530
154530
73390
161950
161950
62466
40893
105325
27762
70805
70805
34529
105372
105372
100093
53760
34911
102557
159129
71520
71520
51167
85292
85292
84701
68078
95356
48844
39893
141322
77731
142708
142708
125336
144487
128616
128616
133015
1

56652
56652
70257
87431
87431
127233
10947
72383
146830
146830
69919
37801
135808
135808
101538
9769
42675
88335
71415
124954
68768
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
132174
91795
59373
148523
30818
75931
15032
114188
47752
109592
159953
37876
37876
28259
62647
37831
46714
46714
156831
74441
85124
124296
99960
40207
68501
147212
78200
10113
146975
100261
100261
47583
154845
108268
108268
64639
128616
128616
151028
162042
70469
128980
76172
6237
156541
144670
108035
108035
108035
31371
36223
47854
49149
45972
61449
139742
77094
77094
40479
43740
152760
72845
72845
71292
69862
148199
51693
146615
146615
159024
159024
67604
75200
37073
62654
88800
33090
94823
39000
60853
116593
87457
87457
31340
145081
114580
47432
119962
134963
113372
84849
151102
149301
39138
147101
152299
43164
42122
141099
141099
53896
135955
111787
111787
86891
108455
81409
85292
85292
68976
80286
78503
78503
113210
152151
55365
55518
142497
53485
147117
146728
87760
122117
6

86152
84294
56319
110403
32438
32438
32438
70913
98832
110748
110748
160827
153918
56453
67568
144423
65702
18096
78880
6144
91823
150157
145594
7477
90228
91313
66282


## Insertar BD

In [ ]:
# getting the number unique users and restaurants
unique_users = reviews_all.user_id.unique().shape[0]
unique_restaurants = reviews_all.business_id.unique().shape[0]
current_reviews= reviews_all[ ["user_id_simple","business_id_simple","stars"] ]
#Create two user-item matrices, one for training and another for testing
matrix_all = np.zeros((unique_users, unique_restaurants))

for line in current_reviews.itertuples():
    #print(line[1])
    matrix_all[line[1], line[2]] = line[3]
matrix_all

In [ ]:
!pip install mysql
!pip install mysql.connector
!pip install mysql-connector-python-rf
import mysql.connector
from mysql.connector import errorcode
#Configuración de la conexión a Mysql
try:
  cnx = mysql.connector.connect(user='user_taller3', password='taller3.', host='127.0.0.1', database='taller3')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

cursor = cnx.cursor()

In [ ]:
sql = "INSERT INTO `recomendations_movies` (`user_id`, `movies_id`, `recomendation_score`) VALUES (%s, %s, %s)"


for i in range(0,unique_users):
    user_id = reviews_all.loc[reviews_all.user_id_simple==i].user_id.unique()[0]
    for j in range(0, unique_restaurants):
        if matrix_all[i][j] == 0:    
            star2=algo1.predict(i, j, r_ui=None, verbose=True)
            #print(star2.est)
            star1=algo2.predict(i, j, r_ui=None, verbose=True)
            #print(star1.est)
            #print(star.est)
            if int(star1.est) == int(star2.est):
                business_id =reviews_all.loc[reviews_all.business_id_simple==j].business_id.unique()[0]
                #print(user_id, business_id)
                print("iguallll")
                cursor.execute(sql, (user_id,business_id,int(star1.est)))
                cnx.commit()